In [ ]:
import yaml
import pandas as pd 
import os

def get_config():
    with open("config.yaml", 'r') as stream:
        config = yaml.safe_load(stream)
    return config

config = get_config()
dir_to = config['directory_to']
file_name_to = config['cleared_data_to']
cleared_data = pd.read_csv(os.path.join(dir_to, file_name_to))
cleared_data.head()

def get_data_info(arr, show=False, threshold=20):
    for ar in arr:
        s = set(ar)
        if show:
            if len(s) < threshold:
                print(len(s), s)
            else:
                print(len(s))
        else:
            print(len(s))

print(cleared_data.shape)

# possible columns for regression analysis
dates = cleared_data['Event Date'].tolist()  #38
categories = cleared_data['Sub-Category'].tolist()  #4
regions = cleared_data['Region'].tolist() #2
pos_category = cleared_data['Positioning Category'].tolist()   #336, nan category included! - according to stats files
pos_sub_category = cleared_data['Positioning Sub-Category'].tolist()  #7785, nan category included!
pack_material = cleared_data['Packaging Material'].tolist()  #23, nan category included!
shelving_details = cleared_data['Shelving Details'].tolist() #3
price_details = cleared_data['Euro Price/Kg'].tolist() #8539 unique, obviously not a category

get_data_info([dates, categories, regions, pos_category, pos_sub_category, pack_material, shelving_details, price_details], show=True, threshold=24)
# clear the nan values

We should not forget to check the amount of non-nan values in the columns considered the undependant variables.
We can use the euro price as an undependant variable for regression analysis but there is significant amount of empty values in columns concerning the price values. 

We can eliminate empty values and predict the future prices or how much the end client is willing to pay for the products on the basis of non empty values.

We should ask about the types of plastic in the Packaging Material column. We could use this info as the independant variable.

We should ask about the Event 6 types - 'Reformulation', 'New Product', 'Import', 'New Package', 'Shelf Snapshots', 'Shelf SnapShots'. Should we predict the most popular categories according to all of the event types or only according to new launches?


Problems:
1) Packweight column is parsed wrong. There are no such packweights as 105 kg. In original data the value constitutes 0.105 kg. Check for the entry with 'Product Id' == 8533553 (the first file provided by IMCD)

2) Values in columns of the string type are not considered empty even if they are. We get wrong statistics. - FIXED


Let's start the regression analysis. Let Sub-Category be the dependant variable, Event Date and Region - undependant.

In [ ]:
products_data = cleared_data[['Event Date', 'Sub-Category', 'Region']].dropna(how='any')
print(products_data.shape)
products_data.head()

Let's convert the date column to the correct datetime format.

In [ ]:
import datetime
import calendar

def month_num(m):
    if len(m) == 1:
        return '0'+m
    return m

months = {month: month_num(str(index)) for index, month in enumerate(calendar.month_abbr) if month}

def convert_to_date(x):
    month, year = x.split()
    return '01-'+months[month] +'-'+ year

products_data['Event Date'] = products_data['Event Date'].map(convert_to_date)
products_data['Event Date'] = pd.to_datetime(products_data['Event Date'], format="%d-%m-%Y")
products_data['Event Date'] = products_data['Event Date'].map(datetime.datetime.toordinal)

Let's represent Sub-Category and Region categories as numbers.

In [ ]:
regions = {'West Europe':0, 'East Europe':1}
categories = {'Savory Biscuits/Crackers':0, 'Sweet Biscuits/Cookies':1, 'Cakes - Pastries & Sweet Goods':2, 'Bread & Bread Products':3}

def categorize_regions(region):
    return regions[region]

def categorize_product_category(category):
    return categories[category]
    
products_data['Sub-Category'] = products_data['Sub-Category'].map(categorize_product_category)
# products_data['Region'] = products_data['Region'].map(categorize_regions)
products_data.head()

In [ ]:
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, CategoricalColorMapper
from bokeh.io import output_notebook
from bokeh.resources import INLINE
output_notebook(INLINE)

source = ColumnDataSource(
        data = {'Event Date': products_data['Event Date'], 'Sub-Category': products_data['Sub-Category'], 'Region': products_data['Region']}
    )

palette=['red', 'blue']
mapper = CategoricalColorMapper(factors=['West Europe', 'East Europe'], palette=palette)
pl = figure(width=900, height=500, title='The relation between the event date and the product category', x_axis_label='Event date', y_axis_label='Sub-Category')
sc = pl.scatter('Event Date', 'Sub-Category', source=source, size=9, alpha = 0.05, color={'field': 'Region', 'transform': mapper}, legend_field='Region')
pl.add_layout(pl.legend[0], 'right')

show(pl)


In [ ]:
from bokeh.models import BasicTicker, ColorBar, LinearColorMapper, PrintfTickFormatter, FixedTicker

products_data_west_europe = products_data[products_data['Region'] == 'West Europe']

source_west_europe = ColumnDataSource(
        data = {'Event Date': products_data_west_europe['Event Date'], 'Sub-Category': products_data_west_europe['Sub-Category'], 'Region': products_data_west_europe['Region']}
    )

p = figure(title="The relation between the event date and the product categories in west europe",
           x_axis_location="above", width=800, height=400,
           toolbar_location='below',
           x_axis_label='Event Date', y_axis_label='Sub-Category')

p.rect(x="Event Date", y="Sub-Category", width=12, height=1,
       source=source_west_europe,
       fill_color={'field': 'Region', 'transform': mapper},
       line_color='black', alpha=0.02)

show(p)

In [ ]:
products_data_east_europe = products_data[products_data['Region'] == 'East Europe']

source_east_europe = ColumnDataSource(
        data = {'Event Date': products_data_east_europe['Event Date'], 'Sub-Category': products_data_east_europe['Sub-Category'], 'Region': products_data_east_europe['Region']}
    )

p = figure(title="The relation between the event date and the product categories in east europe",
           x_axis_location="above", width=800, height=400,
           toolbar_location='below',
           x_axis_label='Event Date', y_axis_label='Sub-Category')

p.rect(x="Event Date", y="Sub-Category", width=12, height=1,
       source=source_east_europe,
       fill_color={'field': 'Region', 'transform': mapper},
       line_color='black', alpha=0.02)

show(p)

In [ ]:
products_data = cleared_data[['Event Date', 'Euro Price/Kg', 'Sub-Category']].dropna(how='any')
print(products_data.shape)

products_data['Event Date'] = products_data['Event Date'].map(convert_to_date)
products_data['Event Date'] = pd.to_datetime(products_data['Event Date'], format="%d-%m-%Y")
products_data['Event Date'] = products_data['Event Date'].map(datetime.datetime.toordinal)
products_data.head()

In [ ]:
bread = products_data[products_data['Sub-Category'] == 'Bread & Bread Products']


source = ColumnDataSource(
        data = {'Event Date': bread['Event Date'], 'Euro Price/Kg': bread['Euro Price/Kg'], 'Sub-Category': bread['Sub-Category']}
    )

palette=['red', 'blue', 'green', 'pink']
mapper = CategoricalColorMapper(factors=['Savory Biscuits/Crackers', 'Sweet Biscuits/Cookies', 'Cakes - Pastries & Sweet Goods', 'Bread & Bread Products'], palette=palette)
pl = figure(width=900, height=500, title='The relation between the event date and the pricing', x_axis_label='Event date', y_axis_label='Euro Price/Kg')
sc = pl.scatter('Event Date', 'Euro Price/Kg', source=source, size=9, alpha = 0.3, color={'field': 'Sub-Category', 'transform': mapper}, legend_field='Sub-Category')
pl.add_layout(pl.legend[0], 'right')

show(pl)

In [ ]:
cleared_data = pd.read_csv(os.path.join(dir_to, file_name_to), parse_dates=['Event Date'])
print('unique event types:', cleared_data['Event'].unique())
cleared_data = cleared_data[['Event Date', 'Sub-Category', 'Event', 'Region']].dropna(how='any')
cleared_data.head()
cleared_data['Event Date'].unique()

In [ ]:
def get_data_for_time_series_analysis(cleared_data, event_type='all', category='all', region='all', add_missing_values=True):
    if event_type != 'all':
        cleared_data  = cleared_data[cleared_data['Event'] == event_type]
    if category != 'all':
        cleared_data  = cleared_data[cleared_data['Sub-Category'] == category]
    if region != 'all':
        cleared_data = cleared_data[cleared_data['Region'] == region]
    cleared_data = cleared_data.groupby(['Event Date']).count() 
    # add missing dates, MS - calendar month begin
    if add_missing_values:
        cleared_data = cleared_data.asfreq(freq='MS')
        # should interpolate? (polynomial or time) or should fill with zeroes?
#         cleared_data = cleared_data.interpolate(method='time', order=2).reset_index()
        cleared_data = cleared_data.fillna(0).reset_index()
    else:
        cleared_data = cleared_data.reset_index() 
    cleared_data = cleared_data[['Event Date', 'Sub-Category']]
    # consider only 2022 year, values in 2023 seem to be unreliable
    return cleared_data[cleared_data['Event Date'] < '2023-01-01']
#     return cleared_data

If we take a look at the date frames we can see that some months are missing for certain categories. We are going to add them for each category and region separately and interpolate. Are missing: 3 months in 2020, 8 months in 2021 for the cakes category (period from 10.2020 to 08.2021 is missing)
Interpolation doesn't work well because a huge period of time is missing. Does it mean that there were no new launches? Should we interpolate it or fill with 0? We could use forward-fill, backward-fill, forecast it using past observations.

In [ ]:
def get_number_of_missing_months(cleared_data, event_type, category, region):
    data = get_data_for_time_series_analysis(cleared_data, event_type=event_type, category=category, region = region, add_missing_values = False)
    print(event_type+':'+category+':'+region+':', len(data), '/', len(cleared_data['Event Date'].unique())-2)
    
get_number_of_missing_months(cleared_data, 'New Product', 'Cakes - Pastries & Sweet Goods', 'West Europe')
get_number_of_missing_months(cleared_data, 'New Product', 'Cakes - Pastries & Sweet Goods', 'East Europe')
get_number_of_missing_months(cleared_data, 'New Product', 'Bread & Bread Products', 'West Europe')
get_number_of_missing_months(cleared_data, 'New Product', 'Bread & Bread Products', 'East Europe')
get_number_of_missing_months(cleared_data, 'New Product', 'Savory Biscuits/Crackers', 'West Europe')
get_number_of_missing_months(cleared_data, 'New Product', 'Savory Biscuits/Crackers', 'East Europe')
get_number_of_missing_months(cleared_data, 'New Product', 'Sweet Biscuits/Cookies', 'West Europe')
get_number_of_missing_months(cleared_data, 'New Product', 'Sweet Biscuits/Cookies', 'East Europe')

get_number_of_missing_months(cleared_data, 'New Product', 'Cakes - Pastries & Sweet Goods', 'all')


In [ ]:
# data for any type of event, for all categories of products in east europe
data1 = get_data_for_time_series_analysis(cleared_data, event_type='New Product')

# data for New Product (new launch) event, for all categories of products in east europe
data2 = get_data_for_time_series_analysis(cleared_data, event_type='New Product', region = 'East Europe')

# data for any type of event, for certain category of products (Bread & Bread Products) in east europe
data3 = get_data_for_time_series_analysis(cleared_data, category='Bread & Bread Products', region = 'East Europe')

# data for New Product (new launch) event, for certain category of products (Bread & Bread Products) in east europe
data4 = get_data_for_time_series_analysis(cleared_data, event_type='New Product', category='Bread & Bread Products', region = 'East Europe')


In [ ]:
import matplotlib.pyplot as plt

# Draw Plot
def plot_df(x, y, title="", xlabel='Date', ylabel='Value'):
    plt.figure(figsize=(16,5))
    plt.plot(x, y, color='tab:red')
    plt.gca().set(title=title, xlabel=xlabel, ylabel=ylabel)
    plt.show()

plot_df(x=data1['Event Date'], y=data1['Sub-Category'], title='Product events between 2020 and 2022 years in East Europe')  

In [ ]:
from scipy.signal import savgol_filter  

def plot_event_lines_for_all_categories_together(cleared_data, region = 'all'):

    data_cakes = get_data_for_time_series_analysis(cleared_data, event_type='New Product', category='Cakes - Pastries & Sweet Goods', region = region)
    data_crackers = get_data_for_time_series_analysis(cleared_data, event_type='New Product', category='Savory Biscuits/Crackers', region = region)
    data_cookies = get_data_for_time_series_analysis(cleared_data, event_type='New Product', category='Sweet Biscuits/Cookies', region = region)
    data_bread = get_data_for_time_series_analysis(cleared_data, event_type='New Product', category='Bread & Bread Products', region = region)

    plt.figure(figsize=(16,10))
    x_filtered = data_cakes[["Sub-Category"]].apply(savgol_filter,  window_length=15, polyorder=3)
    plt.plot(data_cakes['Event Date'], data_cakes['Sub-Category'], label='Cakes', color='red')
    plt.plot(data_cakes['Event Date'], x_filtered, color='red', linestyle='dashed')

    x_filtered = data_crackers[["Sub-Category"]].apply(savgol_filter,  window_length=15, polyorder=3)
    plt.plot(data_crackers['Event Date'], data_crackers['Sub-Category'], label='Savory Biscuits/Crackers', color='green')
    plt.plot(data_crackers['Event Date'], x_filtered, color='green', linestyle='dashed')

    x_filtered = data_cookies[["Sub-Category"]].apply(savgol_filter,  window_length=15, polyorder=3)
    plt.plot(data_cookies['Event Date'], data_cookies['Sub-Category'], label='Sweet Biscuits/Cookies', color='violet')
    plt.plot(data_cookies['Event Date'], x_filtered, color='violet', linestyle='dashed')

    x_filtered = data_bread[["Sub-Category"]].apply(savgol_filter,  window_length=15, polyorder=3)
    plt.plot(data_bread['Event Date'], data_bread['Sub-Category'], label='Bread', color='blue')
    plt.plot(data_bread['Event Date'], x_filtered, color='blue', linestyle='dashed')

    plt.legend(loc='upper right')
    
    if region != 'all':
        title_part = region
    else:
        title_part = 'Europe'

    plt.gca().set(title='Product launches between 2020 and 2022 years in ' + title_part, xlabel='Date', ylabel='Events')
    plt.show()
    
plot_event_lines_for_all_categories_together(cleared_data)
plot_event_lines_for_all_categories_together(cleared_data, region = 'West Europe')
plot_event_lines_for_all_categories_together(cleared_data, region = 'East Europe')

No trends are represented for any category of the products because there is no increasing or decreasing slope observed in the time series. But there may be seasonality - a distinct repeated pattern observed between regular intervals due to seasonal factors. We will check it below.

Another aspect to consider is the cyclic behaviour. It happens when the rise and fall pattern in the series does not happen in fixed calendar-based intervals. For now I would say the behavior of the Sweet Biscuits/Cookies pink line looks wavy with the period of a year and a half. This behaviour is cyclic.

In [ ]:
plot_event_lines_for_all_categories_together(cleared_data, region = 'West Europe')
# graph strongly depends on the interpolation method/filling in with zeroes

Since its a monthly time series let's see if it follows a certain repetitive pattern every year. Let's plot each year as a separate line in the same plot. This lets us compare the year wise patterns side-by-side.

In [ ]:
import numpy as np
import matplotlib as mpl

def plot_seasonal_year_pattern(cleared_data, event_type='New Product', category='all', region = 'all'):

    data_bread = get_data_for_time_series_analysis(cleared_data, event_type=event_type, category=category, region = region)
    data_bread['year'] = [d.year for d in data_bread['Event Date']]
    data_bread['month'] = [d.strftime('%b') for d in data_bread['Event Date']]
    years = data_bread['year'].unique()

    np.random.seed(100)
    mycolors = np.random.choice(list(mpl.colors.XKCD_COLORS.keys()), len(years), replace=False)

    plt.figure(figsize=(16,10))
    for i, y in enumerate(years):
        data = data_bread.loc[data_bread.year==y, :]
        x_filtered = data[["Sub-Category"]].apply(savgol_filter,  window_length=11, polyorder=3)
        plt.plot('month', 'Sub-Category', data=data, color=mycolors[i], label=y)
        plt.text(data_bread.loc[data_bread.year==y, :].shape[0]-.9, data_bread.loc[data_bread.year==y, 'Sub-Category'][-1:].values[0], y, fontsize=12, color=mycolors[i])
        plt.plot(data['month'], x_filtered, color=mycolors[i], linestyle='dashed')


    # Decoration
    plt.gca().set(ylabel='Number of new launches', xlabel='Month')
    plt.yticks(fontsize=12, alpha=.7)
    plt.title("Seasonal Plot of " + category + " for 2020, 2021 and 2022 years in " + region, fontsize=20)
    plt.show()


# plot_seasonal_year_pattern(cleared_data, event_type='New Product', category='Sweet Biscuits/Cookies', region = 'West Europe')
plot_seasonal_year_pattern(cleared_data, event_type='New Product', category='Bread & Bread Products', region = 'West Europe')


On the plot above we can see that there is certain pattern that repeats itself from year to year - the number of launches decreases (or stabilizes) in the late summer- early fall period and grows back in the end of the year for the Bread & Bread Products category of products. Whereas no seasonality can be seen for the Sweet Biscuits/Cookies category.

In [ ]:
import seaborn as sns

data_bread = get_data_for_time_series_analysis(cleared_data, event_type='New Product', category='Bread & Bread Products', region = 'West Europe')
data_bread['year'] = [d.year for d in data_bread['Event Date']]
b=sns.boxplot(x='year', y='Sub-Category', data=data_bread)
plt.title('Year-wise Box Plot(The Trend) for Bread & Bread Products in West Europe')
plt.show()

In [ ]:
from scipy.stats import kde

months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", 
              "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

def plot_monthly_hist(cleared_data, event_type='New Product', category='all', region = 'all', color='green'):
    data_bread = get_data_for_time_series_analysis(cleared_data, event_type=event_type, category=category, region = region)
    data_bread['month'] = [d.strftime('%b') for d in data_bread['Event Date']]
    
    data_bread_gb = data_bread[['month', 'Sub-Category']]
    data_bread_gb = data_bread_gb.groupby(['month']).sum().reset_index()
    data_bread_gb['month'] = pd.Categorical(data_bread_gb['month'], categories=months, ordered=True)
    data_bread_gb.sort_values(by='month', inplace=True) 
    x, y = data_bread_gb['month'], data_bread_gb['Sub-Category']
    plt.bar(x, y, edgecolor="k", color=color)
    plt.title("Distribution of monthly launches of " + category + " for 2020, 2021 and 2022 years in " + region, fontsize=10)
    plt.show()

# plot_monthly_hist(cleared_data, event_type='New Product', category='Bread & Bread Products', region = 'West Europe', color='blue')
# plot_monthly_hist(cleared_data, event_type='New Product', category='Bread & Bread Products', region = 'East Europe', color='pink')

plot_monthly_hist(cleared_data, event_type='New Product', category='Savory Biscuits/Crackers', region = 'West Europe')


In [ ]:
from scipy.signal import savgol_filter  

# in West Europe cake info is missing

data_cakes = get_data_for_time_series_analysis(cleared_data, event_type='New Product', region = 'East Europe')
plt.figure(figsize=(16,10))
x_filtered = data_cakes[["Sub-Category"]].apply(savgol_filter,  window_length=15, polyorder=3)
plt.plot(data_cakes['Event Date'], data_cakes['Sub-Category'], color='red')
plt.plot(data_cakes['Event Date'], x_filtered, color='red', linestyle='dashed')
plt.gca().set(title='Product launches between 2020 and 2022 years in East Europe', xlabel='Date', ylabel='Events')
plt.show()

Let's establish the stationarity of a series. A stationary series is one where the values of the series is not a function of time. It means that the statistical properties of the series like mean, variance and autocorrelation are constant over time. Autocorrelation of the series is the correlation of the series with its previous values.

Following the Augmented Dickey Fuller test (ADH Test) data is stationary if p-value is less than 0.05. 

We can notice that not all datasets are stationary! But the forecasts for a stationary series are more reliable. And autoregressive forecasting models (one of them we are going to use to predict client demand in products) are essentially linear regression models that utilize the lag(s) of the series itself as predictors.

We know that linear regression works best if the predictors (X variables) are not correlated against each other. So, stationarizing the series solves this problem since it removes any persistent autocorrelation, thereby making the predictors(lags of the series) in the forecasting models nearly independent.

So before conducting an analysis we have to make the datasets stationary.

In [ ]:
from statsmodels.tsa.stattools import adfuller, kpss

def is_data_stationary(cleared_data, event_type='New Product', category='all', region = 'all'):
    data = get_data_for_time_series_analysis(cleared_data, event_type=event_type, category=category, region = region)
    
    # ADF Test
    result = adfuller(data['Sub-Category'].values, autolag='AIC')
    return data, result[1] < 0.05

categories['all'] = 5
regions['all'] = 2
for c in categories:
    for r in regions:
        _, is_stationary = is_data_stationary(cleared_data, event_type='New Product', category=c, region = r)
        print(c, r, is_stationary)



The Pearson’s correlation coefficient is a number between -1 and 1 that describes a negative or positive correlation respectively. A value of zero indicates no correlation. The Pearson’s correlation coefficient is depicted as the Y axis.

We can calculate the correlation for time series observations with observations with previous time steps, called lags. Because the correlation of the time series observations is calculated with values of the same series at previous times, this is called a serial correlation, or an autocorrelation.

The plot below shows the lag value along the x-axis and the correlation on the y-axis between -1 and 1. We expect the ACF - AutoCorrelation Function - time series to be strong to a lag of k and the inertia of that relationship would carry on to subsequent lag values, trailing off at some point as the effect was weakened.

Confidence intervals are drawn as a cone. By default, this is set to a 95% confidence interval, suggesting that correlation values outside of this code are very likely a correlation and not a statistical fluke.

In [ ]:
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

def plot_lag_correlation(cleared_data, event_type='New Product', category='all', region = 'all'):
    data = get_data_for_time_series_analysis(cleared_data, event_type=event_type, category=category, region = region)
    plot_acf(data['Sub-Category'])
    plot_pacf(data['Sub-Category'])
    
# is stationary
plot_lag_correlation(cleared_data, event_type='New Product', category='Savory Biscuits/Crackers', region = 'West Europe')

In [ ]:
# is not stationary
plot_lag_correlation(cleared_data, event_type='New Product', category='Bread & Bread Products', region = 'East Europe')

According to the plots it is likely a white noise because there is no correlation between current observations and lags (past observations). It makes it impossible to use the autogressive model...

In [ ]:
from pandas.plotting import lag_plot

def plot_lag_correlation(cleared_data, event_type='New Product', category='all', region = 'all'):
    data = get_data_for_time_series_analysis(cleared_data, event_type=event_type, category=category, region = region)
    fig, axes = plt.subplots(1, 4, figsize=(10,3), sharex=True, sharey=True, dpi=100)
    for i, ax in enumerate(axes.flatten()[:4]):
        lag_plot(data['Sub-Category'], lag=i+1, ax=ax, c='firebrick')
        ax.set_title('Lag ' + str(i+1))

    fig.suptitle('Lag Plots\n(Points get wide and scattered with increasing lag -> lesser correlation)\n', y=1.15)    
    plt.show()
plot_lag_correlation(cleared_data, event_type='New Product', category='Savory Biscuits/Crackers', region = 'West Europe')

In [ ]:
from scipy.stats import entropy

def get_sample_entropy_stats(cleared_data, event_type='New Product', category='all', region = 'all'):
    data = get_data_for_time_series_analysis(cleared_data, event_type=event_type, category=category, region = region)
    return entropy(data['Sub-Category'])

print(get_sample_entropy_stats(cleared_data, event_type='New Product', category='Savory Biscuits/Crackers', region = 'West Europe'))


Let's conduct Granger causality test to see if one time series will be useful to forecast another. The null hypothesis is that the values in the second column can not predict the values in the first column. All p-values that we get are larger than 0.05 so we can not reject the null hypothesis. So it's impossible to say that we can predict the user demand using the month info.

In [ ]:
from statsmodels.tsa.stattools import grangercausalitytests

def is_data_predictable_granger_causality(cleared_data, event_type='New Product', category='all', region = 'all'):
    data = get_data_for_time_series_analysis(cleared_data, event_type=event_type, category=category, region = region)
    data['month'] = data['Event Date'].dt.month

    return grangercausalitytests(data[['Sub-Category', 'month']], maxlag=2)

print(is_data_predictable_granger_causality(cleared_data, event_type='New Product', category='Savory Biscuits/Crackers', region = 'West Europe'))

Now we want to make a forecast for the client demand using ARIMA model. For this purpose we need to define several parameters:

p - the order of the AR term. It refers to the number of lags of Y to be used as predictors;

q - the order of the MA term. It refers to the number of lagged forecast errors that should go into the ARIMA Model;

d - the number of differencing required to make the time series stationary.

Some of the datasets are stationary, for them the d parameter equals 0. Other datasets we have to make stationary. There are several ways to make them stationary:

Differencing the Series (once or more)

Take the log of the series

Take the nth root of the series

Combination of the above

We are going to use differencing technique.

In [ ]:
# Let's work on a data which is not stationary: Bread & Bread Products in east Europe
plt.rcParams.update({'figure.figsize':(9,7), 'figure.dpi':120})

def adf_is_stationary(values):
    result = adfuller(values, autolag='AIC')
    return result[1] < 0.05

def define_differencing_order_plot(cleared_data, event_type='New Product', category='all', region = 'all'):
    data = get_data_for_time_series_analysis(cleared_data, event_type=event_type, category=category, region = region)
    
    fig, axes = plt.subplots(3, 2, sharex=True)
    
    in_data = data['Sub-Category']
    axes[0, 0].plot(in_data)
    axes[0, 0].set_title('Original Series:'+str(adf_is_stationary(in_data.dropna())))
    plot_acf(in_data, ax=axes[0, 1])

    # 1st Differencing
    in_data = in_data.diff()
    axes[1, 0].plot(in_data)
    axes[1, 0].set_title('1st Order Differencing:'+str(adf_is_stationary(in_data.dropna())))
    plot_acf(in_data.dropna(), ax=axes[1, 1])

    # 2nd Differencing
    in_data = in_data.diff()
    axes[2, 0].plot(in_data) 
    axes[2, 0].set_title('2nd Order Differencing:'+str(adf_is_stationary(in_data.dropna())))
    plot_acf(in_data.dropna(), ax=axes[2, 1])

    plt.show()
    
define_differencing_order_plot(cleared_data, event_type='New Product', category='Bread & Bread Products', region = 'East Europe')


For the above series, the time series reaches stationarity with one order of differencing. On looking at the autocorrelation plot for the 2nd differencing the lag goes into the far negative zone fairly quick, which indicates, the series might have been over differenced.

So, for Bread & Bread Products in East Europe we are going to choose the order of differencing as 1 (d parameter in ARIMA model). We will automatically define the order of differencing for other datasets.

In [ ]:
def get_key(event_type, category, region):
    return  f'{event_type}:{category}:{region}'

def make_data_stationary(data):
    in_data = data['Sub-Category'].dropna()
    origin = in_data
    diff_order = 0
    while not adf_is_stationary(in_data):
        in_data = in_data.diff().dropna()
        diff_order += 1
    return in_data, diff_order, origin
        
        
datasets = {}
for c in categories:
    for r in regions:
        data = get_data_for_time_series_analysis(cleared_data, event_type='New Product', category=c, region = r)
        stationary_data, diff_order, origin_data = make_data_stationary(data)
        datasets[get_key('New Product', c, r)] = (stationary_data, diff_order, origin_data)

for k, v in datasets.items():
    print(f'Order of differencing for {k} is {v[1]}')


AR or MA terms are needed to correct any autocorrelation that remains in the differenced series. To determine the AR and MA terms we have to plot the acf (autocorrelation function) and pacf (partial autocorrelation function).

The autocorrelation at lag 1 "propagates" to lag 2 and presumably to higher-order lags. But a partial autocorrelation is the amount of correlation between a variable and a lag of itself that is not explained by correlations at all lower-order-lags aka their mutual correlations.

The rule of thumb is if the partial autocorrelation is significant at lag k and not significant at any higher order lags, i.e. if the PACF "cuts off" at lag k, then this suggests that you should try fitting an autoregressive model of order k (which is parameter p in the ARIBA model). For instance, if the PACF plot has a significant spike only at lag 1 it means that all the higher-order autocorrelations are effectively explained by the lag-1 autocorrelation and the p parameter in ARIBA model should equal to 1 (on the plots below we don't count the first point - the lag-0 - which obviosly correlates to itself with the most possible coefficient - 1).

For example, for Bread & Bread Products in West Europe we can see that the PACF lag 1 is quite significant since it's above the significance line. Lag 2 turns out to be not significant because it falls below the significance limit (blue region). So the p parameter for ARIBA model would be 1.

Just like how we looked at the PACF plot for the number of AR terms, we can look at the ACF plot for the number of MA terms. The ACF tells how many MA terms are required to remove any autocorrelation in the stationarized series. For example, for Sweet Biscuits/Cookies in West Europe the ACF cuts off the lags after lag-2. So the q parameter (MA parameter) for the ARIBA model would be 2.

In [ ]:
for k,v in datasets.items():
    fig, axes = plt.subplots(1, 3, sharex=True)
    axes[0].plot(v[0])
    axes[0].set_title(k, fontsize=6)
    plot_acf(v[0], ax=axes[1])
    plot_pacf(v[0], ax=axes[2])

# according to the ACF and PACF plots define the p and q parameters for the ARIBA model:  
# [p(look at PCAF), q(look at CAF)]      
params = [[0, 0], [3, 0], [1, 1], [1, 2], [0, 0], [1, 1], [0, 0], [0, 0], [0, 0], [1, 1], [0, 0], [1, 1], [0, 0], [2, 1], [0, 0]]
i = 0
print("p, q parameters for ARIBA model:")
datasets_params = {}
for k,v in datasets.items():
    datasets_params[k]=params[i]
    print(k, params[i], '\n')
    i+=1  

The data is stationarized and parameters for ARIBA model are defines. Let's fit the ARIMA model.

The P Values of the AR1 and MA1 terms for New Product:Savory Biscuits/Crackers:West Europe are highly significant (<< 0.05). We also get a line plot of the residual errors, suggesting that there may still be some trend information not captured by the model. And we get a density plot of the residual error values. The distribution of the residual errors shows that indeed there is no bias in the prediction (a zero mean in the residuals) which indicates a good model (really?)

Ideally we would perform this analysis on just the training dataset when developing a predictive model.

Also to define the ideal parameters (p, q) we should try to increase/decrease p and q to see which model gives least AIC and also look for a chart that gives closer actuals and forecasts.


In [ ]:
# we already stationarized the data so we should always pass the d param as 0 and stationarized data 
# or unstationarized data and the actual d parameter

from pandas import DataFrame
from statsmodels.tsa.arima.model import ARIMA

def build_model(key):    
    data_stationarized, d, origin_data = datasets[key]
    p, q = datasets_params[key]

    model = ARIMA(origin_data, order=(p, d, q))

    model_fit = model.fit()
    print(model_fit.summary())

    # residuals + errors density
    residuals = DataFrame(model_fit.resid)
    fig, ax = plt.subplots(1, 2, figsize=(10,3))
    residuals.plot(title="Residuals", ax=ax[0], legend=False)
    residuals.plot(kind='kde', title='Density', ax=ax[1], legend=False)
    plt.show()
    
    # summary stats of residuals
    print(residuals.describe())

In [ ]:
# from [3, 1] optimized to [3, 0] because the p value of the MA1 term was highly insignificant (>> 0.05) which 
# made other coefficient significant (<< 0.05) but did not lower the AIC (it increased from 192 to 196)
build_model('New Product:Savory Biscuits/Crackers:East Europe')

In [ ]:
build_model('New Product:Savory Biscuits/Crackers:West Europe')

In [ ]:
# parameters need to be refines 
for k,v in datasets.items():
    build_model(k)

Let's experiment with the 'New Product:Savory Biscuits/Crackers:West Europe' dataset. We can evaluate an ARIMA model using a walk-forward (the rolling forecast) validation or Out-of-Time Cross validation.

In [ ]:
# evaluate an ARIMA model using a walk-forward validation
import warnings
warnings.filterwarnings("ignore")

from matplotlib import pyplot
from sklearn.metrics import mean_squared_error
from math import sqrt

def plot_walk_forward_validation(key, p=-1, d=-1, q=-1, points_to_predict=5, train_recentage=0.66):
    if train_recentage > 1:
        raise Exepception()
    data_stationarized, d_estimated, origin_data = datasets[key]
    p_estimated, q_estimated = datasets_params[key]
    if p!=-1:
        p_estimated = p
    if q!=-1:
        q_estimated = q
    if d!=-1:
        d_estimated = d

    X = origin_data.values
    size = int(len(X) * train_recentage)
    train, test = X[0:size], X[size:len(X)]
    history = [x for x in train]
    predictions, predictions_new = list(), list()
    print(p_estimated, d_estimated, q_estimated)

    # walk-forward validation
    for t in range(len(test)):
        model = ARIMA(history, order=(p_estimated, d_estimated, q_estimated))
        model_fit = model.fit()
        output = model_fit.forecast()
        predictions.append(output[0])
        predictions_new.append(output[0])
        obs = test[t]
        history.append(obs)
        
    predictions_points = model_fit.predict(start=len(X), end=len(X)+points_to_predict)
    predictions_new = np.append(predictions_new, predictions_points)


    corr = np.corrcoef(predictions, test)[0,1]
    print('correlation:', corr)

    plt.figure(figsize=(12,5))
    pyplot.plot(X, label='actual')
    pyplot.plot(np.append(train, predictions_new), label='forecast_new')
    pyplot.plot(np.append(train, predictions), label='forecast')
    pyplot.plot(train, label='training')

    pyplot.gca().set(title='Walk Forward Validation:'+key)
    pyplot.legend(loc='upper right')
    pyplot.show()
    

The prediction plot looks shifted in relation to original values. This is a standard property of one-step ahead prediction or forecasting (with d=1 order of differencing).

The information used for the forecast is the history up to and including the previous period. A peak, for example, at a period will affect the forecast for the next period, but cannot influence the forecast for the peak period. This makes the forecasts appear shifted in the plot.

A two-step ahead forecast would give the impression of a shift by two periods. Without adding the exogenous regressors or the order of the seasonal component we can not do better.

In [ ]:
plot_walk_forward_validation('New Product:Bread & Bread Products:West Europe', 3,0,3)

In [ ]:
for k,v in datasets.items():
    plot_walk_forward_validation(k)

In [ ]:
def out_of_time_cross_validation_plot(key, p=-1, d=-1, q=-1, points_to_predict=5, train_recentage=0.66):
    data_stationarized, d_estimated, origin_data = datasets[key]
    p_estimated, q_estimated = datasets_params[key]
    if p!=-1:
        p_estimated = p
    if q!=-1:
        q_estimated = q
    if d!=-1:
        d_estimated = d

    X = origin_data.values
    size = int(len(X) * train_recentage)
    train, test = X[0:size], X[size:len(X)]
    
    print(p_estimated, d_estimated, q_estimated)
    model = ARIMA(train, order=(p_estimated, d_estimated, q_estimated))
    model_fit = model.fit()
    
    predictions = model_fit.predict(start=size, end=len(X)+points_to_predict)
    corr = np.corrcoef(predictions[:len(test)], test)[0,1]
    print('correlation:', corr)
    plt.figure(figsize=(12,5))
    plt.plot(X, label='actual')
    plt.plot(np.append(train, predictions), label='forecast_new')
    plt.plot(np.append(train, predictions)[:len(X)], label='forecast')
    plt.plot(train, label='training')
    plt.title('Out of Time Cross Validation:'+ key)
    plt.legend(loc='upper right', fontsize=10)
    plt.show()

In [ ]:
out_of_time_cross_validation_plot('New Product:Bread & Bread Products:West Europe', 3,0,3)

In [ ]:
for k,v in datasets.items():
    out_of_time_cross_validation_plot(k)

In [ ]:
plot_walk_forward_validation('New Product:Sweet Biscuits/Cookies:East Europe', 2, 0, 2)
out_of_time_cross_validation_plot('New Product:Sweet Biscuits/Cookies:East Europe', 2, 0, 2)

In [ ]:
plot_walk_forward_validation('New Product:Bread & Bread Products:West Europe', 3,0,3)
out_of_time_cross_validation_plot('New Product:Bread & Bread Products:West Europe', 3,0,3)

Let's choose the best p, d, q parameters for ARIMA model automatically. Best model means lowest AIC value.

In [ ]:
import pmdarima as pm

def choose_best_params_automatically(key):
    print(key)
    _, _, origin_data = datasets[key]
    model = pm.auto_arima(origin_data.values, start_p=0, start_q=0,
                          test='adf',       # use adftest to find optimal 'd'
                          max_p=5, max_q=5, # maximum p and q
                          m=1,              # frequency of series
                          d=None,           # let model determine 'd'
                          seasonal=False,   # No Seasonality
                          start_P=0, 
                          D=0, 
                          trace=True,
                          error_action='ignore',  
                          suppress_warnings=True, 
                          stepwise=True)

#     print(model.summary())
choose_best_params_automatically('New Product:Sweet Biscuits/Cookies:East Europe')

In [ ]:
plot_walk_forward_validation('New Product:Sweet Biscuits/Cookies:East Europe', 1, 0, 0)
out_of_time_cross_validation_plot('New Product:Sweet Biscuits/Cookies:East Europe', 1, 0, 0)

In [ ]:
def walk_forward_validation_for_all_datasets():
    for k,v in datasets.items():
        choose_best_params_automatically(k)

In [ ]:
walk_forward_validation_for_all_datasets()
params = {'New Product:Savory Biscuits/Crackers:West Europe': [0,1,1],
         'New Product:Savory Biscuits/Crackers:East Europe': [3,2,0],
         'New Product:Savory Biscuits/Crackers:all': [0,1,1],
         'New Product:Sweet Biscuits/Cookies:West Europe': [3,2,0],
         'New Product:Sweet Biscuits/Cookies:East Europe': [1,0,0],
         'New Product:Sweet Biscuits/Cookies:all': [1,2,0],
         'New Product:Cakes - Pastries & Sweet Goods:West Europe': [0,2,1],
         'New Product:Cakes - Pastries & Sweet Goods:East Europe': [0,1,0],
         'New Product:Cakes - Pastries & Sweet Goods:all': [3,2,0],
         'New Product:Bread & Bread Products:West Europe': [0,1,3],
         'New Product:Bread & Bread Products:East Europe': [1,2,0],
         'New Product:Bread & Bread Products:all': [1,1,1],
         'New Product:all:West Europe': [1,2,0],
         'New Product:all:East Europe': [3,2,0],
         'New Product:all:all': [1,2,0]}


In [ ]:
for key, pars in params.items():
    plot_walk_forward_validation(key, pars[0], pars[1], pars[2])

In [ ]:
for key, pars in params.items():
    out_of_time_cross_validation_plot(key, pars[0], pars[1], pars[2])